In [1]:
import os
from dotenv import load_dotenv
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage
from langchain.callbacks.tracers import LangChainTracer
from langchain_anthropic import ChatAnthropic
from langchain import PromptTemplate

load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
#model = ChatAnthropic(model="claude-3-haiku-20240307", anthropic_api_key=anthropic_api_key)
model = ChatAnthropic(model="claude-3-5-sonnet-20240620", anthropic_api_key=anthropic_api_key, max_tokens=4096)
# Set up LangChain tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_PROJECT"] = "note-taker"

# State

In [2]:
class NoteTakerState(TypedDict):
    company_data: str	
    project_general: str	
    project_requirements: str		
    project_sprint_state : str
    project_backlog : str
    employee_profiles: str	
    meeting_history: str	
    meeting_type: str		
    transcript: str
    note_taker_halucinate : bool
    reflection_haluconate : bool
    note_draft : str
    note_after_reflection : str
    note_final : str


# Utils

In [24]:
import json

def initialize_NoteTaker_state(company_data: str, project_general: str, project_requirements: str, project_sprint_state: str, project_backlog :str, employee_profiles: str, meeting_history: str, meeting_type: str, transcript : str ) -> NoteTakerState:
    return NoteTakerState(
    	company_data=company_data,
        project_general = project_general,
    	project_requirements = project_requirements,	
    	project_sprint_state= project_sprint_state,	
        project_backlog = project_backlog,
    	employee_profiles=employee_profiles,
        meeting_history = meeting_history,	
    	meeting_type=meeting_type,			
    	transcript=transcript,
        note_taker_halucinate = False,
        reflection_haluconate = False,
        note_draft = "",
        note_after_reflection = "",
        note_final = "",
    )

def load_txt_to_str(file_path):
    with open(file_path, 'r', encoding='utf-8') as txt_file:
        text_content = txt_file.read()
    return text_content


def load_markdown_to_str(file_path):
    with open(file_path, 'r', encoding='utf-8') as md_file:
        markdown_content = md_file.read()
    return markdown_content

def load_and_format_prompt(prompt_path, **kwargs):
    with open(prompt_path, 'r', encoding='utf-8') as file:
        template = file.read()
    
    prompt = PromptTemplate(
        input_variables=list(kwargs.keys()),
        template=template
    )
    
    formatted_prompt = prompt.format(**kwargs)
    
    return formatted_prompt

def export_state(state, folder_path):
    # Create the folder if it doesn't exist
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    id = len(files) + 1
    
    filename = f"state_log_{id}.json"
    
    os.makedirs(folder_path, exist_ok=True)
    
    # Construct the full file path
    file_path = os.path.join(folder_path, filename)
    
    # Write the state dict to a JSON file
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(state, f, indent=4)
        
def update_markdown_file(file_path, new_content):
    """
    Update a markdown file with new content.

    Args:
    file_path (str): The path to the markdown file to be updated.
    new_content (str): The new content to be written to the file.

    Returns:
    None
    """
    try:
        with open(file_path, 'w', encoding='utf-8') as md_file:
            md_file.write(new_content)
        print(f"Successfully updated {file_path}")
    except IOError as e:
        print(f"An error occurred while updating {file_path}: {e}")


# Prompts and nodes

In [4]:
state = initialize_NoteTaker_state(
    company_data=load_markdown_to_str(file_path="../data/company-data/company-data.md"),
    project_general=load_markdown_to_str(file_path= "../data/company-data/current-project-general.md"),
    project_requirements=load_markdown_to_str(file_path="../data/company-data/current-project-requirements.md"),
    project_sprint_state=load_markdown_to_str(file_path="../data/company-data/current-project-sprint-state.md"),
    project_backlog=load_markdown_to_str(file_path="../data/company-data/current-project-backlog.md"),
    employee_profiles=load_markdown_to_str(file_path="../data/company-data/employee-profiles.md"),
    meeting_history=load_markdown_to_str(file_path="../data/company-data/current-meeting-history.md"),
    meeting_type="Sprint Planning",
    transcript=load_txt_to_str(file_path="../data/transcript-workflow/1.txt"),
)
state

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [32]:
NOTE_TAKER_PROMPT = """
You are a Scrum master in the team.
You have meetings and to keep everything organized you make general notes after each meeting.This will help you and your team to work more efficiently.
However, it is essential to get this accurate. Every valuable piece of information that has been said needs to be included.
You don't want to oversimplify things - thus leave out valuable information. 
You just want to transform the transcript of the meeting to a readable format and leave out the recurring information. 
In this way, the note can be used as a valuable asset.

To get you started you will find some information about the company and the project. 
This will help you to get context about the state of the project.
Here you can find information about the company in general: \n #### \n {company_data} \n #### \n 
Here you can find information about the employees and their skilles and responsibilities. \n #### \n {employee_profiles} \n #### \n 
Here you can find information about the current project that the team is working on, in general: \n #### \n {project_general} \n #### \n
Here you can find information about the project requirements : \n #### \n {project_requirements} \n #### \n 

Note that the outcome of the current meeting hasnt been used to update the state. 
This state represents the state of the project before the meeting happened.
Here you can find information about the state of the project: \n #### \n {project_sprint_state} \n #### \n  
Here you can find information about the state of the backlog: \n #### \n {project_backlog} \n #### \n

To help you better understand the timeline and status of the project, you'll find information here about previous meetings.
Please note that the current meeting is already included in the document.
Past meeting: \n #### \n {meeting_history} \n #### \n

Here you will find the transcript of the meeting. Use this to make a note.
Transcript: \n #### \n {transcript} \n #### \n

IMPORTANT: 
Do not leave information out that might be valuable. 
Remember your task is not to make a summary thus oversimplify and reduce information by compressing it but to write out that has been said and transform the meeting transcript into readable format.
You should only response with the structured note.
When you are finished say FINISHED
"""

In [7]:
def note_taker_node(state: NoteTakerState) -> NoteTakerState:
    formatted_prompt = NOTE_TAKER_PROMPT.format(
      company_data = state.get("company_data"),
      employee_profiles = state.get("employee_profiles"),
      project_general = state.get("project_general"),
      project_requirements = state.get("project_requirements"),
      project_sprint_state = state.get("project_sprint_state"),
      project_backlog = state.get("project_backlog"),
      meeting_history = state.get("meeting_history"),
      transcript = state.get("transcript"),
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Generate a meeting note based on the provided information.")
	]
    response = model.invoke(messages)
    state["note_draft"] = response.content
    print(state["note_draft"])
    return state

In [8]:
state = note_taker_node(state=state)

Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Forbidden"}')


Meeting Note: HealthTrack Pro Sprint Planning Meeting

Date: [Date not specified in the transcript]
Time: [Time not specified in the transcript]
Attendees: Sarah Chen (Project Manager/Scrum Master), Alex Rodriguez (Senior Full-Stack Developer), Emily Watson (Frontend Developer), Michael Kim (Backend Developer), Olivia Martinez (QA Engineer/DevOps Specialist), Liam Foster (UI/UX Designer)

1. Project Overview
   - HealthTrack Pro: A comprehensive web application for personal health management
   - Key features: Activity tracking, nutrition logging, health metrics dashboard, goal setting, recommendations, social features, integration with fitness devices/apps

2. Sprint Goals
   - Sprint duration: 2 weeks
   - Story point target: 65 points
   - Sprint goal: "Implement core user authentication and health dashboard functionality, laying the foundation for HealthTrack Pro's MVP"

3. Prioritized Features for MVP
   a. User Authentication (31 story points)
      - Backend (13 points): User mo

Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Forbidden"}')
Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Forbidden"}')
Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Forbidden"}')
Failed to batch ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/batch in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/batch', '{

In [9]:
NOTE_APPROVAL_PROMPT = """
You are a Scrum master in the team.
You have meetings and to keep everything organized you make general notes after each meeting.This will help you and your team to work more efficiently.
However, it is essential to get this accurate. Every valuable piece of information that has been said needs to be included.
You have made the first draft from the meeting. 
Now you have to make sure that the note contain only what has been said, without adding anything that hasn't been said.

To get you started you will find some information about the company and the project. 
This will help you to get context about the state of the project.
Here you can find information about the company in general: \n #### \n {company_data} \n #### \n 
Here you can find information about the employees and their skilles and responsibilities. \n #### \n {employee_profiles} \n #### \n 
Here you can find information about the current project that the team is working on, in general: \n #### \n {project_general} \n #### \n
Here you can find information about the project requirements : \n #### \n {project_requirements} \n #### \n 

Note that the outcome of the current meeting hasnt been used to update the state. 
This state represents the state of the project before the meeting happened.
Here you can find information about the state of the project: \n #### \n {project_sprint_state} \n #### \n  
Here you can find information about the state of the backlog: \n #### \n {project_backlog} \n #### \n

To help you better understand the timeline and status of the project, you'll find information here about previous meetings.
Please note that the current meeting is already included in the document.
Past meeting: \n #### \n {meeting_history} \n #### \n

Here is your first draft: \n #### \n {note_draft} \n #### \n

Finally, here you will find the transcript of the meeting. Use this to verify the note doesnt contain false information.
Transcript: \n #### \n {transcript} \n #### \n

IMPORTANT: 
If you find anything in the notes that is false or was not mentioned during the meeting or in previous documents, respond with: NOTE CONTAINS FALSE INFO.
If everything in the notes is accurate and was mentioned during the meeting or in previous documents, respond with: NOTE CONTAINS VALID INFO
"""

In [10]:
def note_approvel_node(state: NoteTakerState) -> NoteTakerState:
    formatted_prompt = NOTE_APPROVAL_PROMPT.format(
      company_data = state.get("company_data"),
      employee_profiles = state.get("employee_profiles"),
      project_general = state.get("project_general"),
      project_requirements = state.get("project_requirements"),
      project_sprint_state = state.get("project_sprint_state"),
      project_backlog = state.get("project_backlog"),
      meeting_history = state.get("meeting_history"),
      transcript = state.get("transcript"),
      note_draft = state.get("note_draft")
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Validate the node based on the provided information.")
	]
    response = model.invoke(messages)
    if "NOTE CONTAINS FALSE INFO" in response.content:
        state["note_taker_halucinate"] = True
        print(response.content)
    else:
        state["note_taker_halucinate"] = False
        print(response.content)
    return state

In [11]:
state = note_approvel_node(state=state)

NOTE CONTAINS VALID INFO

The meeting note accurately reflects the content of the transcript and aligns with the information provided in the previous documents. It includes the key points discussed during the sprint planning meeting, such as the project overview, sprint goals, prioritized features for MVP, technical considerations and risks, and action items. The note also correctly captures the team members' contributions and the estimates for each feature. There are no apparent discrepancies or false information in the meeting note.


In [12]:
REFLECTION_PROMPT = """
You are a Scrum master reviewing your own note-taking work.
Your task is to critically analyze the notes you've taken from the team meeting and ensure they accurately represent the discussion without adding or omitting important information.

To get you started you with this task find some information about the company and the project. 
This will help you to get context about the state of the project.
Here you can find information about the company in general: \n #### \n {company_data} \n #### \n 
Here you can find information about the employees and their skilles and responsibilities. \n #### \n {employee_profiles} \n #### \n 
Here you can find information about the current project that the team is working on, in general: \n #### \n {project_general} \n #### \n
Here you can find information about the project requirements : \n #### \n {project_requirements} \n #### \n 

Note that the outcome of the current meeting hasnt been used to update the state. 
This state represents the state of the project before the meeting happened.
Here you can find information about the state of the project: \n #### \n {project_sprint_state} \n #### \n  
Here you can find information about the state of the backlog: \n #### \n {project_backlog} \n #### \n

To help you better understand the timeline and status of the project, you'll find information here about previous meetings.
Please note that the current meeting is already included in the document.
Past meeting: \n #### \n {meeting_history} \n #### \n


Here is your first draft:

{note_draft}

Now, reflect on this draft and consider the following:

Accuracy: Have you included all valuable information from the transcript without adding anything that wasn't explicitly discussed in the meeting?
Completeness: Did you capture every important point mentioned in the meeting? Are there any parts of the discussion that you might have overlooked?
Clarity: Is the note in a readable format? Have you successfully transformed the transcript into a clear, organized document?
Relevance: Have you included only information relevant to the project and team? Did you successfully leave out recurring or unnecessary information?
Context: Did you use the provided company, project, and employee information appropriately to provide context without inserting it into the meeting notes as if it was discussed?
Structure: Is the note structured in a way that makes it easy for team members to find and use the information?
Objectivity: Have you maintained objectivity in your note-taking, avoiding personal interpretations or assumptions?
Action Items: Have you clearly identified all action items, assignments, and deadlines mentioned in the meeting?

Based on this reflection, identify any areas where your notes could be improved.
Be specific about what changes need to be made and why. Remember, your goal is to create an accurate, comprehensive, and useful record of the meeting without adding information that wasn't discussed or omitting valuable points.
If you find areas for improvement, please create a revised version of the notes addressing these issues. 
If you believe your original draft accurately represents the meeting discussion, explain why you think no changes are necessary.
IMPORTANT:
Your reflection and any revisions should focus solely on accurately representing the content of the meeting transcript. 
Do not add new information or interpretations that were not explicitly stated in the meeting.
When you have completed your reflection and any necessary revisions, please state "REFLECTION COMPLETED".
"""

In [13]:
def reflection_node(state: NoteTakerState) -> NoteTakerState:
    formatted_prompt = REFLECTION_PROMPT.format(
      company_data = state.get("company_data"),
      employee_profiles = state.get("employee_profiles"),
      project_general = state.get("project_general"),
      project_requirements = state.get("project_requirements"),
      project_sprint_state = state.get("project_sprint_state"),
      project_backlog = state.get("project_backlog"),
      meeting_history = state.get("meeting_history"),
      transcript = state.get("transcript"),
      note_draft = state.get("note_draft")
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Make the second version of the draft based on the provided information.")
    ]
    response = model.invoke(messages)

    state["note_after_reflection"] = response.content
    print(state["note_after_reflection"])
    return state

In [14]:
state

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [15]:
state = reflection_node(state=state)
state

Thank you for the instruction. I'll create a revised version of the meeting notes based on the provided information and my reflections on the initial draft. Here's the revised version:

Meeting Note: HealthTrack Pro Sprint Planning Meeting

Date: June 19, 2023
Time: 10:00 AM - 12:30 PM
Attendees: Sarah Chen (Project Manager/Scrum Master), Alex Rodriguez (Senior Full-Stack Developer), Emily Watson (Frontend Developer), Michael Kim (Backend Developer), Olivia Martinez (QA Engineer/DevOps Specialist), Liam Foster (UI/UX Designer)

1. Sprint Overview
   - Sprint duration: 2 weeks
   - Story point target: 65 points
   - Sprint goal: "Implement core user authentication and health dashboard functionality, laying the foundation for HealthTrack Pro's MVP"

2. Prioritized Features for Sprint
   a. User Authentication (31 story points)
      - Backend (13 points): User model, authentication logic, API endpoints
      - Frontend (10 points): Registration/login forms, validation, authentication sta

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [18]:
def reflection_approvel_node(state: NoteTakerState) -> NoteTakerState:
    formatted_prompt = NOTE_APPROVAL_PROMPT.format(
      company_data = state.get("company_data"),
      employee_profiles = state.get("employee_profiles"),
      project_general = state.get("project_general"),
      project_requirements = state.get("project_requirements"),
      project_sprint_state = state.get("project_sprint_state"),
      project_backlog = state.get("project_backlog"),
      meeting_history = state.get("meeting_history"),
      transcript = state.get("transcript"),
      note_draft = state.get("note_after_reflection")
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Validate the note based on the provided information.")
	]
    response = model.invoke(messages)
    if "NOTE CONTAINS FALSE INFO" in response.content:
        state["note_taker_halucinate"] = True
        print(response.content)
    else:
        state["note_taker_halucinate"] = False
        state["note_final"] = state["note_after_reflection"]
        print(response.content)
    return state

In [19]:
reflection_approvel_node(state=state)

NOTE CONTAINS VALID INFO


{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n

In [22]:
export_state(state=state, folder_path="../data/notes")

In [23]:
UPDATE_PROJECT_STATE = """
You are a Scrum master in the team.
You have meetings and to keep everything organized you have made general notes after each meeting.
This will help you and your team to work more efficiently.
Now your task is to update the project sprint status document by the notes.
However, it is essential to get this accurate. Every valuable piece of information that has been said needs to be included.
You don't want to oversimplify things - thus leave out valuable information. 
In this way, the document representing the current status of the project can be used as a valuable asset.

To get you started you will find some information about the company and the project. 
This will help you to get context about the state of the project.
Here you can find information about the company in general: \n #### \n {company_data} \n #### \n 
Here you can find information about the employees and their skilles and responsibilities. \n #### \n {employee_profiles} \n #### \n 
Here you can find information about the current project that the team is working on, in general: \n #### \n {project_general} \n #### \n
Here you can find information about the project requirements : \n #### \n {project_requirements} \n #### \n 

To help you better understand the timeline and status of the project, you'll find information here about previous meetings.
Please note that the current meeting is already included in the document.
Past meeting: \n #### \n {meeting_history} \n #### \n

Here you will find your notes about the meeting. Use this to update the sprint status document.
Note: \n #### \n {note_final} \n #### \n

Here is the content of the current state. This document will updated: \n #### \n {project_sprint_state} \n #### \n  

Use this template to create the updated version. \n #### \n {template} \n #### \n  


IMPORTANT: 
Do not leave information out that might be valuable. 
Do not write down information that hasn't been said.
Use the template writtend in markdown.
You should only response with the structured update.
"""

In [44]:
def update_project_state(state: NoteTakerState) -> str:
    formatted_prompt = UPDATE_PROJECT_STATE.format(
        company_data = state.get("company_data"),
		employee_profiles = state.get("employee_profiles"),
		project_general = state.get("project_general"),
		project_requirements = state.get("project_requirements"),
		meeting_history = state.get("meeting_type"),
		note_final = state.get("note_final"),
		project_sprint_state = state.get("project_sprint_state"),
		template = load_markdown_to_str(file_path="../data/company-data/project-sprint-state-template.md"),
	)
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Update the state")
	]
    response = model.invoke(messages)
    update_markdown_file(file_path="../data/company-data/current-project-sprint-state.md", new_content=response.content)
    print(response.content)
    return state
	


In [28]:
update_project_state(state=state)

Successfully updated ../data/company-data/current-project-sprint-state.md


'# Project Sprint State Template\n\n## Project Name: HealthTrack Pro\n\n### Sprint Information\n- Sprint Number: 1\n- Sprint Goal: "Implement core user authentication and health dashboard functionality, laying the foundation for HealthTrack Pro\'s MVP"\n- Sprint Duration: June 20, 2023 to July 3, 2023\n- Total Working Days: 10\n\n### Team Capacity\n- Team Members: 6\n- Total Capacity (Story Points): 65\n- Committed Capacity (Story Points): 99\n\n### Sprint Backlog\n| User Story | Story Points | Status | Assigned To | Notes |\n|------------|--------------|--------|-------------|-------|\n| User Authentication - Backend | 13 | Not Started | Michael Kim | User model, authentication logic, API endpoints |\n| User Authentication - Frontend | 10 | Not Started | Emily Watson | Registration/login forms, validation, authentication state management |\n| User Authentication - Security | 5 | Not Started | Olivia Martinez | Password hashing, rate limiting, security audits |\n| User Authentication -

In [41]:
def create_meeting_summary_workflow(checkpointer):
	workflow = StateGraph(NoteTakerState)

	# Define nodes
	workflow.add_node("note_taker", note_taker_node)
	workflow.add_node("note_approvel", note_approvel_node)
	workflow.add_node("reflection", reflection_node)
	workflow.add_node("reflection_approvel", reflection_approvel_node)
	workflow.add_node("update_project_state", update_project_state)
	
	# Define edges
	workflow.add_edge("note_taker", "note_approvel")
	workflow.add_conditional_edges(
		"note_approvel",
		lambda x: "reflection" if not x.get("note_taker_halucinate") else "note_taker",
		{
			"reflection": "reflection",
			"note_taker": "note_taker"
		}
	)
	workflow.add_edge("reflection", "reflection_approvel")
	workflow.add_conditional_edges(
	    "reflection_approvel",
	    lambda x: "reflection" if x.get("reflection_haluconate") else "update_project_state",
	    {
	        "reflection": "reflection",
	        "update_project_state": "update_project_state"
	    }
	)
	workflow.add_edge("update_project_state", END)


	# Set entry point
	workflow.set_entry_point("note_taker")

	return workflow.compile(checkpointer = checkpointer)

In [56]:
# todo
def run_stage_one():
	state = initialize_NoteTaker_state(
    	company_data=load_markdown_to_str(file_path="../data/company-data/company-data.md"),
    	project_general=load_markdown_to_str(file_path= "../data/company-data/current-project-general.md"),
    	project_requirements=load_markdown_to_str(file_path="../data/company-data/current-project-requirements.md"),
    	project_sprint_state=load_markdown_to_str(file_path="../data/company-data/current-project-sprint-state.md"),
    	project_backlog=load_markdown_to_str(file_path="../data/company-data/current-project-backlog.md"),
    	employee_profiles=load_markdown_to_str(file_path="../data/company-data/employee-profiles.md"),
    	meeting_history=load_markdown_to_str(file_path="../data/company-data/current-meeting-history.md"),
    	meeting_type="Sprint 1 Review and Retrospective",
    	transcript=load_txt_to_str(file_path="../data/transcript-workflows/SprintReviewandRetrospective6.txt"),
	)
	all_states = []

	with SqliteSaver.from_conn_string(":memory:") as memory:
		thread = {"configurable": {"thread_id": "1"}}
		graph = create_meeting_summary_workflow(memory)
		for s in graph.stream(state, thread):
			all_states.append(s)
		last_state = {key: value for key, value in all_states[-1].items()}
		export_state(state=last_state["update_project_state"], folder_path="../data/logs")
	return last_state

In [30]:
with SqliteSaver.from_conn_string(":memory:") as memory:
    graph = create_meeting_summary_workflow(memory)
    print(graph.get_graph().draw_mermaid())

%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	note_taker(note_taker)
	note_approvel(note_approvel)
	reflection(reflection)
	reflection_approvel(reflection_approvel)
	update_project_state(update_project_state)
	__end__([<p>__end__</p>]):::last
	__start__ --> note_taker;
	note_taker --> note_approvel;
	reflection --> reflection_approvel;
	update_project_state --> __end__;
	note_approvel -.-> reflection;
	note_approvel -.-> note_taker;
	reflection_approvel -.-> reflection;
	reflection_approvel -.-> update_project_state;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [57]:
last_state = run_stage_one()


Meeting Notes: Sprint 1 Review and Retrospective - HealthTrack Pro

Date: July 3, 2023
Time: 2:00 PM - 4:00 PM
Attendees: Sarah Chen (PM), Alex Rodriguez (Senior Full-Stack Developer), Emily Watson (Frontend Developer), Michael Kim (Backend Developer), Olivia Martinez (QA/DevOps Specialist), Liam Foster (UI/UX Designer)

1. Sprint 1 Overview
   - Sprint Goal: Implement core user authentication and health dashboard functionality for HealthTrack Pro's MVP
   - Focus areas: User authentication (frontend and backend), health dashboard development, development pipeline setup

2. Sprint 1 Accomplishments

   2.1 User Authentication System
   
   Backend (Michael Kim):
   - Completed user model and core authentication logic
   - Implemented functional API endpoints for registration and login
   - Integrated JSON Web Tokens for secure session management
   - Implemented indexing to improve database query performance
   - Pending: Password reset functionality

   Frontend (Emily Watson):
   - I